## Training file paired with LambdaAutoTraining project

In [1]:
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow import keras
import numpy as np
import boto3
import os
import time
import decimal

**Set environment variables**

In [2]:
table_models = os.environ.get('TABLE_MODELS')
if not table_models:
    table_models = 'lambdaautotraining-models-dev'
    
table_data = os.environ.get('TABLE_DATA')
if not table_data:
    table_data = 'lambdaautotraining-data-dev'
    
bucket = os.environ.get('BUCKET')
if not bucket:
    bucket = 'lambdaautotraining-dev'
    
region = os.environ.get('REGION')
if not region:
    region = 'us-east-1'
    
print(table_models, table_data, bucket, region)

lambdaautotraining-models-dev lambdaautotraining-data-dev lambdaautotraining-dev us-east-1


In [3]:
dynamodb = boto3.resource('dynamodb', region_name=region)
dynamodb_data = dynamodb.Table(table_data)
dynamodb_models = dynamodb.Table(table_models)

**Retrieve all of the data from DynamoDB**

In [4]:
response = dynamodb_data.scan()
response_data = response['Items']

while 'LastEvaluatedKey' in response:
    response = dynamodb_data.scan(
        ExclusiveStartKey=response['LastEvaluatedKey']
    )
    response_data += response['Items']

In [5]:
len(response_data)

8

**Convert datatypes for training**

In [6]:
x = []
y = []

In [7]:
for rec in response_data:
    # convert Decimal datatype to float
    x.append([
        float(rec['temp']),
        float(rec['clo']),
        float(rec['rh']),
        float(rec['wind']),        
    ])
    
    # one-hot encoded
    y_rec = [0,0,0]
    if rec['label'] == 'cold': y_rec[0] = 1
    elif rec['label'] == 'ok': y_rec[1] = 1
    elif rec['label'] == 'warm': y_rec[2] = 1
    y.append(y_rec)

In [8]:
x_array = np.array(x)
y_array = np.array(y)

**Execute training**

In [9]:
tf.keras.backend.clear_session()

In [10]:
model = keras.Sequential()
model.add(keras.layers.Dense(8, input_shape=[4], activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [11]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [31]:
model.fit(
    x_array, y_array, 
    epochs=25, batch_size=4
)

Epoch 1/25
8/8 [==============================] - 0s 452us/sample - loss: 0.0371 - acc: 1.0000
Epoch 2/25
8/8 [==============================] - 0s 469us/sample - loss: 0.0343 - acc: 1.0000
Epoch 3/25
8/8 [==============================] - 0s 434us/sample - loss: 0.0306 - acc: 1.0000
Epoch 4/25
8/8 [==============================] - 0s 389us/sample - loss: 0.0278 - acc: 1.0000
Epoch 5/25
8/8 [==============================] - 0s 451us/sample - loss: 0.0261 - acc: 1.0000
Epoch 6/25
8/8 [==============================] - 0s 422us/sample - loss: 0.0242 - acc: 1.0000
Epoch 7/25
8/8 [==============================] - 0s 446us/sample - loss: 0.0231 - acc: 1.0000
Epoch 8/25
8/8 [==============================] - 0s 425us/sample - loss: 0.0221 - acc: 1.0000
Epoch 9/25
8/8 [==============================] - 0s 388us/sample - loss: 0.0207 - acc: 1.0000
Epoch 10/25
8/8 [==============================] - 0s 434us/sample - loss: 0.0195 - acc: 1.0000
Epoch 11/25
8/8 [==============================] 

In [37]:
model.history.history['loss'][-1:][0]

0.010748700704425573

**Convert model for js and upload**

In [25]:
created = int(time.time()*1000)
print(created)

1569976905088


In [13]:
tfjs.converters.save_keras_model(model, './tf_js/')

In [26]:
for f in os.listdir('./tf_js'):
    boto3.client('s3').upload_file('./tf_js/'+f, bucket, 'models/'+str(created)+'/'+f)
    boto3.client('s3').upload_file('./tf_js/'+f, bucket, 'models/latest/'+f)

**Update model data in dynamodb**

In [44]:
dynamodb_models.put_item(
    Item={
        'created': created,
        'samples': decimal.Decimal(str(len(response_data))),
        'max_data_created': decimal.Decimal(str(max([int(i['created']) for i in response_data]))),
        'loss': decimal.Decimal(str(model.history.history['loss'][-1:][0])),
        'accuracy': decimal.Decimal(str(model.history.history['acc'][-1:][0])),
    }
)

{'ResponseMetadata': {'RequestId': 'D9SU3PREN1S4G0CE3MP558OKU3VV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Wed, 02 Oct 2019 00:50:05 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'D9SU3PREN1S4G0CE3MP558OKU3VV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}